In [59]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge, LinearRegression




In [152]:
data = pd.read_csv('cfs_2012_pumf_csv.txt')
data.shape
data.head()

,SHIPMT_ID,ORIG_STATE,ORIG_MA,ORIG_CFS_AREA,DEST_STATE,DEST_MA,DEST_CFS_AREA,NAICS,QUARTER,SCTG,MODE,SHIPMT_VALUE,SHIPMT_WGHT,SHIPMT_DIST_GC,SHIPMT_DIST_ROUTED,TEMP_CNTL_YN,EXPORT_YN,EXPORT_CNTRY,HAZMAT,WGT_FACTOR
0,1,25,148,25-148,25,148,25-148,333,2,35,14,2178,11,14,17,N,N,N,N,208.5
1,2,42,428,42-428,6,41740,06-41740,311,3,35,14,344,11,2344,2734,N,N,N,N,193.3
2,3,26,220,26-220,47,314,47-314,322,2,27,4,4197,5134,470,579,N,N,N,N,51.2
3,4,20,556,20-556,20,556,20-556,323,1,29,4,116,6,3,3,N,N,N,N,238.7
4,5,12,99999,12-99999,12,99999,12-99999,4235,3,33,5,388,527,124,201,N,N,N,N,398.1


## Featuring Engineering

In [183]:
# transform NAICS values into 6 major categories: mining (except oil & gas), manufacturing, 
# wholesale, storage, fuel, online shopping, publishing, administration
def NAICS(x):
    if x >= 300 and x <= 400:
        return 'Manufacturing'
    elif x >= 4200 and x <= 4300:
        return 'Wholesale'
    elif x == 212:
        return 'Mining'
    elif x == 4541:
        return 'Online shopping'
    elif x == 45431:
        return 'Fuel'
    elif x == 4931:
        return 'Storage'
    elif x == 5111:
        return 'Publishing'
    else:
        return 'Administration'
    
data['NAICS_type'] = data['NAICS'].map(lambda x: NAICS(x))
data['NAICS_type'].value_counts()

Manufacturing      2149701
Wholesale          1882220
Mining              140933
Storage             101644
Fuel                101200
Online shopping      86257
Publishing           53137
Administration       32569
Name: NAICS_type, dtype: int64

In [196]:
# transform MODE values into 4 major categories:
def MODE(x):
    if x >= 3 and x <= 5:
        return 'Truck'
    elif x == 6:
        return 'Rail'
    elif x >= 7 and x <= 10 or x == 101:
        return 'Water'
    elif x == 11:
        return 'Air'
    elif x == 12:
        return 'Pipeline'
    elif x == 14:
        return 'Parcel'
    elif x == 15:
        return 'Truck and rail'
    else:
        return 'Other/multiple'

data['transportation_type'] = data['MODE'].map(lambda x: MODE(x))
data['transportation_type'].value_counts()
# data['MODE'].value_counts()

Truck             3231969
Parcel            1165297
Air                 68809
Rail                38458
Truck and rail      19070
Other/multiple      16489
Water                3896
Pipeline             3673
Name: transportation_type, dtype: int64

In [93]:
y = data[['SHIPMT_VALUE']]
X = data[['SHIPMT_WGHT', 'SHIPMT_DIST_ROUTED']]

# 'NAICS', 'QUARTER', 'MODE',

In [94]:
X

,SHIPMT_WGHT,SHIPMT_DIST_ROUTED
0,11,17
1,11,2734
2,5134,579
3,6,3
4,527,201
...,...,...
4547656,133,152
4547657,29887,683
4547658,137,16
4547659,1240,22


In [125]:
X_train , X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

linreg = LinearRegression()
linreg.fit(X_train, y_train)
print(linreg.coef_)
print(linreg.intercept_)

[[0.10908507 3.18528677]]
[12604.92542866]
